In [1]:
# Parse a line of weather station data, returning the average wind direction measurement 
#
import re
def parse(line):
    match = re.search("Dm=(\d+)", line)
    if match:
        val = match.group(1)
        return [int(val)]
    return []

In [2]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

#Create SparkSession
spark = (
    SparkSession.builder
    .appName("streamSpark")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

sc=spark.sparkContext

23/01/08 15:44:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/08 15:44:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
ssc=StreamingContext(sc, 1)

In [4]:
lines = ssc.socketTextStream("rtd.hpwren.ucsd.edu", 12028)

In [5]:
vals = lines.flatMap(parse)

In [6]:
window = vals.window(10, 5)

In [7]:
def stats(rdd):
    print(rdd.collect())
    if rdd.count() > 0:
        print("max = {} min = {}".format(rdd.max(), rdd.min()))

In [8]:
window.foreachRDD(stats)

In [12]:
sc.setLogLevel("ERROR")
ssc.start()

[99, 99, 98, 98, 97, 96, 95, 94, 97, 101]
max = 101 min = 94


[96, 95, 94, 97, 101, 105, 108, 112, 115]
max = 115 min = 94


[105, 108, 112, 115, 118, 120, 117, 119, 121, 120]
max = 121 min = 105


[118, 120, 117, 119, 121, 120, 118, 116, 116, 116, 116]
max = 121 min = 116


[118, 116, 116, 116, 116, 116, 116, 116, 116, 116]
max = 118 min = 116


[116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116]
max = 116 min = 116


[116, 116, 116, 116, 116, 116, 111, 110, 110, 110, 110]
max = 116 min = 110


In [13]:
ssc.stop()

23/01/08 15:50:20 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

In [ ]:
######
#https://towardsdatascience.com/a-fast-look-at-spark-structured-streaming-kafka-f0ff64107325
######


from pyspark.sql import SparkSession

KAFKA_BOOTSTRAP_SERVERS = "kafka:9092"
KAFKA_TOPIC = "test_topic"

spark = SparkSession.builder.appName("read_test_straeam").getOrCreate()

# Reduce logging
spark.sparkContext.setLogLevel("WARN")

df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", KAFKA_TOPIC) \
    .option("startingOffsets", "earliest") \
    .load()

df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .writeStream \
    .format("console") \
    .outputMode("append") \
    .start() \
    .awaitTermination()